In [1]:
import importlib
import os
import sys
import polars as pl

current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, ".."))
sys.path.append(parent_dir)

import shared.data
import shared.data.loaders.gp
import shared.data.loaders.wsc

2026-01-26 20:54:49.789 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2026-01-26 20:54:49.792 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2026-01-26 20:54:49.793 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2026-01-26 20:54:49.793 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2026-01-26 20:54:49.794 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2026-01-26 20:54:49.795 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2026-01-26 20:54:49.796 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2026-01-26 20:54:49.797 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2026-01-

In [2]:
importlib.reload(shared.data)
importlib.reload(shared.data.loaders.gp)
importlib.reload(shared.data.loaders.wsc)

wsc = shared.data.loaders.wsc.load_wsc(csv_directory="../data/raw/wsc")
combined_df = shared.data.loaders.gp.load_gp(csv_directory="../data/processed/gp", output_csv=None)

flattened = shared.data.create_flat_dataset(combined_df, metric="points")

2026-01-26 20:54:52.410 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2026-01-26 20:54:52.414 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2026-01-26 20:54:52.415 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2026-01-26 20:54:52.416 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2026-01-26 20:54:52.417 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2026-01-26 20:54:52.419 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2026-01-26 20:54:52.420 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2026-01-26 20:54:52.421 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2026-01-

In [3]:
def attempted_mapping(wsc_df, gp_df):
    """Update a WSC dataset with identifiers from a GP dataset."""
    gp_index = gp_df.select(["Name", "Country", "Nick", "user_pseudo_id"]).unique()
    gp_index = gp_index.with_columns(pl.col("Name").str.to_lowercase().alias("name_lc"))

    wsc_mapped = wsc_df.with_columns(
        pl.col("Name").str.to_titlecase().str.replace(",", "").alias("Name"),
        pl.col("Name").str.to_lowercase().str.replace(",", "").alias("name_lc")
    )

    wsc_mapped = wsc_mapped.with_columns(
        pl.col("Name")
        .str.replace_all(",", "")
        .str.split(" ")
        .list.reverse()
        .list.join(" ")
        .alias("flipped_name")
    )

    joined = wsc_mapped.join(gp_index, on="Name", how="left").drop("name_lc_right")

    joined_flipped = (
        joined
            .join(gp_index, left_on="flipped_name", right_on="Name", how="left")
            .drop(["Country_right", "Nick_right"])
            .rename({"user_pseudo_id_right": "user_pseudo_id_flipped_name"})
    )

    wsc_and_gp = (
        joined_flipped
            .join(gp_index, on="name_lc", how="left")
            .rename(({"user_pseudo_id_right": "user_pseudo_id_name_lc"}))
    )

    # Use our attempted identifiers in a hierarchy by name, then flipped name, then lowercase name
    wsc_and_gp = wsc_and_gp.with_columns(
        pl.col("user_pseudo_id")
        .fill_null(pl.col("user_pseudo_id_flipped_name"))
        .fill_null(pl.col("user_pseudo_id_name_lc"))
        .alias("matched_id")
    )

    manual_map = shared.competitions.WSC_NAME_TO_GP_ID_OVERRIDE

    expr = None
    for key, value in manual_map.items():
        if expr is None:
            expr = pl.when(pl.col("Name") == key).then(pl.lit(value))
        else:
            expr = expr.when(pl.col("Name") == key).then(pl.lit(value))
        expr = expr.when(pl.col("flipped_name") == key).then(pl.lit(value))
    expr = expr.otherwise(pl.col("matched_id")).alias("matched_id")
    wsc_and_gp = (
        wsc_and_gp
            .with_columns(expr)
            .drop("user_pseudo_id")
            .rename({"matched_id": "user_pseudo_id"})
    )

    wsc_and_gp = wsc_and_gp.with_columns(pl.col("user_pseudo_id").alias("matched_id"))

    # TODO: Add in country
    wsc_and_gp = wsc_and_gp.with_columns(
        pl.col("user_pseudo_id").fill_null(pl.col("Name"))
    )

    return wsc_and_gp

wsc_matched_names = attempted_mapping(wsc, flattened)

no_matches = wsc_matched_names.filter(pl.col("matched_id").is_null())
print(f"No matches for {len(no_matches)} users of {len(wsc_matched_names)}")
display(no_matches.sort(by=["Official_rank"]).head(10))

No matches for 498 users of 2224


year,Name,Official,Official_rank,Unofficial_rank,WSC_total,WSC_entry,WSC_t1 points,WSC_t2 points,WSC_t3 points,WSC_t4 points,WSC_t5 points,WSC_t6 points,WSC_t7 points,WSC_t8 points,WSC_t9 points,WSC_t10 points,WSC_t11 points,WSC_t12 points,WSC_t13 points,WSC_t14 points,WSC_t15 points,WSC_t16 points,name_lc,flipped_name,Country,Nick,user_pseudo_id_flipped_name,name_lc_right,Name_right,Country_right,Nick_right,user_pseudo_id_name_lc,user_pseudo_id,matched_id
i32,str,bool,i64,i64,f64,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str
2024,"""Rick Uppelschoten""",false,null,81,1107.0,true,165.0,70.0,120.0,110.0,80.0,102.0,125.0,null,null,70.0,265.0,null,null,null,null,null,"""rick uppelschoten""","""Uppelschoten Rick""",null,null,null,null,null,null,null,null,"""Rick Uppelschoten""",null
2024,"""Cheng Hao""",false,null,104,875.0,true,75.0,80.0,85.0,60.0,120.0,0.0,115.0,null,null,120.0,220.0,null,null,null,null,null,"""cheng hao""","""Hao Cheng""",null,null,null,null,null,null,null,null,"""Cheng Hao""",null
2024,"""Bauernschmitt Katharina""",false,null,111,710.0,true,115.0,40.0,30.0,65.0,30.0,0.0,170.0,null,null,40.0,220.0,null,null,null,null,null,"""bauernschmitt katharina""","""Katharina Bauernschmitt""",null,null,null,null,null,null,null,null,"""Bauernschmitt Katharina""",null
2024,"""Yao Guangxu""",false,null,116,621.0,true,135.0,40.0,55.0,0.0,60.0,26.0,120.0,null,null,20.0,165.0,null,null,null,null,null,"""yao guangxu""","""Guangxu Yao""",null,null,null,null,null,null,null,null,"""Yao Guangxu""",null
2024,"""Joffrey Duc""",false,null,123,581.0,true,125.0,40.0,0.0,80.0,35.0,46.0,95.0,null,null,45.0,115.0,null,null,null,null,null,"""joffrey duc""","""Duc Joffrey""",null,null,null,null,null,null,null,null,"""Joffrey Duc""",null
2024,"""Li Jiahui""",false,null,131,480.0,true,35.0,40.0,50.0,0.0,0.0,0.0,115.0,null,null,65.0,175.0,null,null,null,null,null,"""li jiahui""","""Jiahui Li""",null,null,null,null,null,null,null,null,"""Li Jiahui""",null
2024,"""Gao Jihong""",false,null,136,420.0,true,0.0,40.0,55.0,40.0,30.0,0.0,50.0,null,null,45.0,160.0,null,null,null,null,null,"""gao jihong""","""Jihong Gao""",null,null,null,null,null,null,null,null,"""Gao Jihong""",null
2024,"""Efe Alan""",false,null,137,400.0,true,80.0,25.0,25.0,65.0,50.0,0.0,0.0,null,null,25.0,130.0,null,null,null,null,null,"""efe alan""","""Alan Efe""",null,null,null,null,null,null,null,null,"""Efe Alan""",null
2024,"""Tigran Dhunjishaw Wadia""",false,null,138,400.0,true,45.0,25.0,30.0,65.0,85.0,0.0,30.0,null,null,45.0,75.0,null,null,null,null,null,"""tigran dhunjishaw wadia""","""Wadia Dhunjishaw Tigran""",null,null,null,null,null,null,null,null,"""Tigran Dhunjishaw Wadia""",null


In [4]:
no_matches.filter(pl.col("year") == 2022)

year,Name,Official,Official_rank,Unofficial_rank,WSC_total,WSC_entry,WSC_t1 points,WSC_t2 points,WSC_t3 points,WSC_t4 points,WSC_t5 points,WSC_t6 points,WSC_t7 points,WSC_t8 points,WSC_t9 points,WSC_t10 points,WSC_t11 points,WSC_t12 points,WSC_t13 points,WSC_t14 points,WSC_t15 points,WSC_t16 points,name_lc,flipped_name,Country,Nick,user_pseudo_id_flipped_name,name_lc_right,Name_right,Country_right,Nick_right,user_pseudo_id_name_lc,user_pseudo_id,matched_id
i32,str,bool,i64,i64,f64,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str
2022,"""Pranav Kamesh S""",true,27,32,1300.0,true,180.0,90.0,135.0,60.0,200.0,100.0,60.0,null,null,260.0,115.0,100.0,null,null,null,null,"""pranav kamesh s""","""S Kamesh Pranav""",null,null,null,null,null,null,null,null,"""Pranav Kamesh S""",null
2022,"""Adam Šuráň""",false,null,48,1120.0,true,150.0,55.0,110.0,90.0,175.0,125.0,100.0,null,null,50.0,195.0,70.0,null,null,null,null,"""adam šuráň""","""Šuráň Adam""",null,null,null,null,null,null,null,null,"""Adam Šuráň""",null
2022,"""Sunghwa Hong""",true,45,53,1060.0,true,150.0,55.0,155.0,30.0,50.0,75.0,80.0,null,null,180.0,175.0,110.0,null,null,null,null,"""sunghwa hong""","""Hong Sunghwa""",null,null,null,null,null,null,null,null,"""Sunghwa Hong""",null
2022,"""Jaipal Reddy M""",true,52,63,985.0,true,120.0,30.0,155.0,70.0,25.0,155.0,100.0,null,null,180.0,90.0,60.0,null,null,null,null,"""jaipal reddy m""","""M Reddy Jaipal""",null,null,null,null,null,null,null,null,"""Jaipal Reddy M""",null
2022,"""Edouard Lebeau""",false,null,70,950.0,true,120.0,75.0,180.0,30.0,0.0,90.0,50.0,null,null,225.0,120.0,60.0,null,null,null,null,"""edouard lebeau""","""Lebeau Edouard""",null,null,null,null,null,null,null,null,"""Edouard Lebeau""",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2022,"""Kaja Sõstra""",true,147,183,175.0,true,0.0,20.0,25.0,0.0,0.0,0.0,0.0,null,null,50.0,70.0,10.0,null,null,null,null,"""kaja sõstra""","""Sõstra Kaja""",null,null,null,null,null,null,null,null,"""Kaja Sõstra""",null
2022,"""Sena Başaran""",false,null,183,175.0,true,0.0,10.0,20.0,0.0,0.0,0.0,10.0,null,null,75.0,60.0,0.0,null,null,null,null,"""sena başaran""","""Başaran Sena""",null,null,null,null,null,null,null,null,"""Sena Başaran""",null
2022,"""Ömer Tuna Barutçu""",false,null,187,125.0,true,30.0,0.0,0.0,0.0,0.0,30.0,10.0,null,null,25.0,20.0,10.0,null,null,null,null,"""ömer tuna barutçu""","""Barutçu Tuna Ömer""",null,null,null,null,null,null,null,null,"""Ömer Tuna Barutçu""",null


In [5]:
import difflib

import pandas as pd

no_matches_subset = no_matches.filter(pl.col("year") == 2022)
#no_matches_subset = no_matches_subset.to_pandas()
display(no_matches_subset)

# Function to find the closest match for each name in list1
def find_closest_match(list1, list2):
    closest_matches = {}
    for name in list1:
        closest = difflib.get_close_matches(name, list2, n=1, cutoff=0.0)
        closest_matches[name] = closest[0] if closest else None
    return closest_matches

# Get closest matches
matches = find_closest_match(no_matches_subset['Name'], flattened["Name"])

matches

for wsc_name, gp_name in matches.items():
    gp_id = flattened.filter(pl.col("Name") == gp_name).select("user_pseudo_id").item(0,0)
    print(f"\"{wsc_name}\": \"{gp_id}\",")

year,Name,Official,Official_rank,Unofficial_rank,WSC_total,WSC_entry,WSC_t1 points,WSC_t2 points,WSC_t3 points,WSC_t4 points,WSC_t5 points,WSC_t6 points,WSC_t7 points,WSC_t8 points,WSC_t9 points,WSC_t10 points,WSC_t11 points,WSC_t12 points,WSC_t13 points,WSC_t14 points,WSC_t15 points,WSC_t16 points,name_lc,flipped_name,Country,Nick,user_pseudo_id_flipped_name,name_lc_right,Name_right,Country_right,Nick_right,user_pseudo_id_name_lc,user_pseudo_id,matched_id
i32,str,bool,i64,i64,f64,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str
2022,"""Pranav Kamesh S""",true,27,32,1300.0,true,180.0,90.0,135.0,60.0,200.0,100.0,60.0,null,null,260.0,115.0,100.0,null,null,null,null,"""pranav kamesh s""","""S Kamesh Pranav""",null,null,null,null,null,null,null,null,"""Pranav Kamesh S""",null
2022,"""Adam Šuráň""",false,null,48,1120.0,true,150.0,55.0,110.0,90.0,175.0,125.0,100.0,null,null,50.0,195.0,70.0,null,null,null,null,"""adam šuráň""","""Šuráň Adam""",null,null,null,null,null,null,null,null,"""Adam Šuráň""",null
2022,"""Sunghwa Hong""",true,45,53,1060.0,true,150.0,55.0,155.0,30.0,50.0,75.0,80.0,null,null,180.0,175.0,110.0,null,null,null,null,"""sunghwa hong""","""Hong Sunghwa""",null,null,null,null,null,null,null,null,"""Sunghwa Hong""",null
2022,"""Jaipal Reddy M""",true,52,63,985.0,true,120.0,30.0,155.0,70.0,25.0,155.0,100.0,null,null,180.0,90.0,60.0,null,null,null,null,"""jaipal reddy m""","""M Reddy Jaipal""",null,null,null,null,null,null,null,null,"""Jaipal Reddy M""",null
2022,"""Edouard Lebeau""",false,null,70,950.0,true,120.0,75.0,180.0,30.0,0.0,90.0,50.0,null,null,225.0,120.0,60.0,null,null,null,null,"""edouard lebeau""","""Lebeau Edouard""",null,null,null,null,null,null,null,null,"""Edouard Lebeau""",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2022,"""Kaja Sõstra""",true,147,183,175.0,true,0.0,20.0,25.0,0.0,0.0,0.0,0.0,null,null,50.0,70.0,10.0,null,null,null,null,"""kaja sõstra""","""Sõstra Kaja""",null,null,null,null,null,null,null,null,"""Kaja Sõstra""",null
2022,"""Sena Başaran""",false,null,183,175.0,true,0.0,10.0,20.0,0.0,0.0,0.0,10.0,null,null,75.0,60.0,0.0,null,null,null,null,"""sena başaran""","""Başaran Sena""",null,null,null,null,null,null,null,null,"""Sena Başaran""",null
2022,"""Ömer Tuna Barutçu""",false,null,187,125.0,true,30.0,0.0,0.0,0.0,0.0,30.0,10.0,null,null,25.0,20.0,10.0,null,null,null,null,"""ömer tuna barutçu""","""Barutçu Tuna Ömer""",null,null,null,null,null,null,null,null,"""Ömer Tuna Barutçu""",null


"Pranav Kamesh S": "Pranav Kamesh (pranavmanu) - India",
"Adam Šuráň": "Adam Suran (hjkl18) - Czech Rep.",
"Sunghwa Hong": "SeongHwa Hong (Nickless) - Korea, South",
"Jaipal Reddy M": "Jaipal Reddy (mjaipal) - India",
"Edouard Lebeau": "Édouard Lebeau (mandourin) - France",
"Pál Madarassy": "Aniko Madarassy (Aniko) - Hungary",
"Hanur Kim": "Pear Kim (Nickless) - Korea, South",
"Rick Uppelschoten": "Kaari Helstein (pisipaha) - Estonia",
"Tiit-Hendrik Piibeleht": "Tiit Hendrik Piibeleht (Tiiduke) - Estonia",
"Martina Prinerova": "Martina Prinerová (eifel_mp) - Slovakia",
"Tom Groot Kormelink": "Tomáš Kruml (Nickless) - Czech Rep.",
"Radoslav Bochev": "Rado Bo4ev (tres4o) - Bulgaria",
"Dávid Braun": "David Braun (Braunka625) - Hungary",
"Natalie Grabowsky": "Natalia Sakowska (Nickless) - Poland",
"Athanasios Darkadakis": "Thanos Darkadakis (sakito) - Greece",
"François Blecon": "François STRICH (frst) - France",
"Michaele Bongiorno": "michele bongiorno (wiseman89) - Italy",
"Antonis Lalat

In [ ]:
for wsc_name, gp_name in matches.items():
    print(gp_name)
    #gp_name = matches[wsc_name]
    #gp_id = flattened[flattened["Name"] == gp_name]["user_pseudo_id"].iloc[0]
    flattened.filter(pl.col("Name") == gp_name).select("user_pseudo_id").item(0,0))
    break

Pranav Kamesh
Pranav Kamesh (pranavmanu) - India


In [15]:
flattened[flattened["Name"].str.contains("lacoste")]

,user_pseudo_id,2014_1,2014_2,2014_3,2014_4,2014_5,2014_6,2014_7,2015_1,2015_2,...,2024_2,2024_3,2024_4,2024_5,2024_6,2024_7,2024_8,Name,Nick,Country


In [31]:
no_matches["year"].value_counts()

year
2018    81
2017    61
2010    57
2011    55
2012    50
2019    39
2015    39
2014    33
2023    27
2024    26
Name: count, dtype: int64